In [44]:
pip install moviepy speechrecognition opencv-python pytesseract selenium pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 388 kB 4.0 MB/s eta 0:00:01
     |████████████████████████████████| 32.8 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 54.8 MB 2.5 MB/s eta 0:00:012
     |████████████████████████████████| 78 kB 5.4 MB/s eta 0:00:011
     |████████████████████████████████| 64 kB 5.5 MB/s eta 0:00:011
     |████████████████████████████████| 313 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 4.6 MB/s eta 0:00:01
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110744 sha256=44f1f05e437771cde2c71494c2e79cbb4049036460810bc09763fdd9bc985cb6
  Stored in directory: /Users/nadyashechtman/Library/Caches/pip/wheels/29/15/e4/4f790bec6acd51a00b67e8ee1394f0bc6e0135c315f8ff399a
  Created wheel for imageio-ffmpeg: filename=imageio_ffmpeg-0.5.1-py3-none-any.whl size=167

In [34]:
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [69]:
##start OCR(Get Frames)

##Get names from screen recordings of names
import cv2

video_path = 'VideoEZ.MP4'

vidcap = cv2.VideoCapture(video_path)

frame_count = 0
saved_count = 0
frame_interval = 10  # Save every 5th frame

success, image = vidcap.read()

while success:
    if frame_count % frame_interval == 0:
        cv2.imwrite(f"frame{saved_count}.jpg", image)
        saved_count += 1  # Increment the count of saved frames
    success, image = vidcap.read()
    frame_count += 1  # Increment the count of total frames


In [70]:
print(frame_count)

4782


In [ ]:
##READ OCR
import pytesseract
from PIL import Image


final_set = set()
final_dict = {}
ID = 0
actual_range = frame_count//frame_interval


for i in range(actual_range):
    frame_path = f'frame{i}.jpg'
    img = Image.open(frame_path)
    text = pytesseract.image_to_string(img)
    final_set.add(text)
    final_dict[text] = ID
    ID+=1
    print(f"Text in frame {i}: {text}")


##Copy Text and use LLM to clean the text. Get the names now. 

In [ ]:
import re 

names = ["example hello", "example 2hello","example h3ello","exa4mple hello","exa5mple hello"]
result = re.split(r'\d+.', names)

# Remove newline characters from each element in the list and exclude empty strings
result = [res.replace("\n", "").strip() for res in result if res.strip()]

# Split each element by the first hyphen
result = [res.split('-', 1) for res in result]



print(result)
len(result)


In [ ]:
import pandas as pd

df = pd.DataFrame(result, columns=['Names', 'Affiliation'])

df['Affiliation'] = df['Affiliation'].fillna("")

df['linkedin'] = ""
print(df)

names_list = df["Names"].tolist()

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

final_name_dict = {}
driver = webdriver.Chrome()

driver.get("https://www.linkedin.com/login")

time.sleep(2)

email_field = driver.find_element(By.ID, "username")
email_field.send_keys("your email to log into LinkedIn") 

password_field = driver.find_element(By.ID, "password")
password_field.send_keys("Your password to log into Linkedin") 

login_button = driver.find_element(By.XPATH, "//button[@type='submit']")
login_button.click()

time.sleep(5)
for name in names_list:
    search_bar = driver.find_element(By.XPATH, "//input[@placeholder='Search']")
    search_bar.clear()
    search_bar.send_keys(name)  
    search_bar.send_keys(Keys.RETURN)  

    time.sleep(5)

    first_profile = driver.find_element(By.XPATH, "(//a[contains(@href, '/in/')])[1]")
    first_profile.click() 

    profile_url = driver.current_url
    final_name_dict[name] = profile_url
    
    print("Profile URL:", profile_url)

# Close the browser
driver.quit()

Profile URL: https://www.linkedin.com/search/results/all/?keywords=slava%20shechtman&origin=GLOBAL_SEARCH_HEADER&sid=lNs
Profile URL: https://www.linkedin.com/in/eyal-shechtman/
Profile URL: https://www.linkedin.com/search/results/all/?keywords=Ron%20Hoory&origin=GLOBAL_SEARCH_HEADER&sid=G%2C_
Profile URL: https://www.linkedin.com/search/results/all/?keywords=Moshe%20Levi&origin=GLOBAL_SEARCH_HEADER&sid=T%3AG


In [ ]:
import pandas as pd



# List of names
fakeData = ["example hello", "example 2hello","example h3ello","exa4mple hello","exa5mple hello"]

# Corresponding affiliations
Affi = ["Company A", "Company B", "Company C", "Company D"]

# Create the DataFrame
df = pd.DataFrame({
    'Name': fakeData,
    'Affiliation': Affi
})

# Initialize the LinkedIn column
df["LinkedIn"] = ""

# Add LinkedIn URLs using a loop
id = 1
for name in fakeData:
    df.loc[df['Name'] == name, "LinkedIn"] = "linkedin.com/" + str(id)
    id += 1

# Display the DataFrame
print(df)

In [ ]:
##Use the output above with the Profile URL to create your dictionary of Names and Linkedins
##makes the dict into a DF and df.to_csv(xyz, index = False) will be able to make your csv file